- Import das bibliotecas utilizadas (pandas, openpyxl, geopy)
- Caso alguma delas esteja faltando use o comando: `pip install nome_lib`

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from tkinter.filedialog import askopenfilename
from geopy.geocoders import Nominatim
import pandas as pd

- Lendo a base de dados
- Aqui você pode fazer de várias maneiras, importante é gerar um DataFrame com as colunas que serão trabalhadas
    - Aqui estou pedindo ao usuário que escolha o arquivo excel com as informações

In [ ]:
df = pd.read_excel(askopenfilename()).fillna("")

- Achei por bem fazer os seguintes tratamentos:
    - 1°: Uma nova coluna com o nome da rua apenas pra usar como chave primária
    - 2°: Gerar um novo DataFrame para ter liberdade com os dados
    - 3°: Criar uma coluna com o endereço completo de forma mais limpa (sem espaços em branco e vírgulas desnecessárias)
    - 4°: Reduzir a base, removendo duplicadas, antes de realizar a consulta.

In [ ]:
df["nome_rua"] = df.Endereço.apply(lambda e: e.split(",")[0].strip())

new_df = pd.DataFrame(df[["nome_rua", "Bairro", "Cidade", "Estado"]])

new_df["full_address"] = new_df.apply(lambda row: ", ".join(filter(None, [row["Bairro"], row["Cidade"], row["Estado"]])).strip(), axis=1)

new_df.drop_duplicates("full_address", inplace=True)

- Aqui a mágica acontece: crio o geo localizador e aplico ele na coluna com o endereço completo que criei antes
- Depois separo o resultado em novas colunas

In [ ]:
geolocator = RateLimiter(Nominatim(user_agent="deskrobot").geocode, min_delay_seconds=1)
new_df["location"] = new_df["full_address"].apply(geolocator)

new_df["latitude"] = new_df["location"].apply(lambda loc: loc.point.latitude if loc else None)
new_df["longitude"] = new_df["location"].apply(lambda loc: loc.point.longitude if loc else None)

- Uno as duas bases com os dados tratados e salvo em um novo arquivo excel.

In [ ]:
df_full_address = df.merge(new_df[["nome_rua", "location", "latitude", "longitude", "full_address"]], on="nome_rua")
df_full_address.drop("endereco_maps", axis=1).fillna("").to_excel("coordenadas.xlsx", index=False)